In [1]:
from rich import print
import logging

logging.basicConfig(level=logging.DEBUG)  # TODO set to INFO

## Data from [Daily Surface Weather and Climatological Summaries (Daymet)](https://daymet.ornl.gov/)


### Data at a single point


In [2]:
from springtime.datasets import Daymet

source = Daymet(
    points=[
        [-84.2625, 36.0133],
        [-86, 39.6],
        [-85, 40],
    ],
    years=[2000, 2002],
)

source.load()

INFO:springtime.datasets.daymet:Found /home/peter/.cache/springtime/daymet_single_point_-84.2625_36.0133_2000_2002.csv
INFO:springtime.datasets.daymet:Found /home/peter/.cache/springtime/daymet_single_point_-86.0_39.6_2000_2002.csv
INFO:springtime.datasets.daymet:Found /home/peter/.cache/springtime/daymet_single_point_-85.0_40.0_2000_2002.csv


,datetime,geometry,dayl,prcp,srad,swe,tmax,tmin,vp
0,2000-01-01,POINT (-84.26250 36.01330),34571.11,0.00,316.51,1.05,17.50,2.30,720.50
1,2000-01-02,POINT (-84.26250 36.01330),34606.19,0.00,258.33,0.00,19.34,6.69,980.37
2,2000-01-03,POINT (-84.26250 36.01330),34644.13,16.00,170.64,0.00,22.19,11.40,1347.53
3,2000-01-04,POINT (-84.26250 36.01330),34684.93,30.74,169.79,0.00,16.64,6.60,974.48
4,2000-01-05,POINT (-84.26250 36.01330),34728.55,0.00,168.51,0.00,4.19,-2.22,518.74
...,...,...,...,...,...,...,...,...,...
1090,2002-12-27,POINT (-85.00000 40.00000),33017.90,0.00,191.33,0.00,-1.32,-8.52,321.14
1091,2002-12-28,POINT (-85.00000 40.00000),33041.86,0.00,241.06,0.00,3.25,-8.11,331.72
1092,2002-12-29,POINT (-85.00000 40.00000),33069.32,4.20,176.20,0.00,5.59,-5.09,418.27
1093,2002-12-30,POINT (-85.00000 40.00000),33100.26,5.68,187.65,0.00,11.74,-2.24,518.04


In [3]:
from springtime.datasets import Daymet

source = Daymet(
    variables=["tmin", "tmax"],
    area={"name": "indianapolis", "bbox": [-86.5, 39.5, -86, 40.1]},
    years=[2000, 2002],
    frequency="monthly",
)

source.load()

DEBUG:springtime.utils:Executing R code:
        library(daymetr)
        daymetr::download_daymet_ncss(
            location = c(40.1,-86.5,39.5,-86.0),
            start = 2000,
            end =  2002,
            frequency = "monthly",
            param = c('tmin','tmax'),
            mosaic = "na",
            path = "/home/peter/.cache/springtime/daymet_bbox_indianapolis_monthly")
        


  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


,datetime,geometry,tmin,tmax
0,2000-01-16 12:00:00,POINT (-86.48421 40.16066),-9.039355,0.901290
1,2000-01-16 12:00:00,POINT (-86.47208 40.15913),-9.057420,0.886129
2,2000-01-16 12:00:00,POINT (-86.45995 40.15761),-9.066451,0.878710
3,2000-01-16 12:00:00,POINT (-86.44782 40.15609),-9.076129,0.871935
4,2000-01-16 12:00:00,POINT (-86.43569 40.15456),-9.085161,0.866129
...,...,...,...,...
131035,2002-12-16 12:00:00,POINT (-86.05566 39.44629),-3.248387,4.527742
131036,2002-12-16 12:00:00,POINT (-86.04368 39.44472),-3.230645,4.546774
131037,2002-12-16 12:00:00,POINT (-86.03169 39.44315),-3.215806,4.563226
131038,2002-12-16 12:00:00,POINT (-86.01971 39.44158),-3.210968,4.559678


In [4]:
source._box_dir

PosixPath('/home/peter/.cache/springtime/daymet_bbox_indianapolis_monthly')

In [2]:
# The latitude of Washington, the USA is 47.751076, and the longitude is -120.740135
target_lat = 47.751076
target_lon = -120.740135

In [3]:
# Create a dataset instance
dataset = DaymetSinglePoint(point=(target_lon, target_lat), years=[1990, 2020])
dataset

DaymetSinglePoint(dataset='daymet_single_point', years=YearRange(start=1990, end=2020), variables=(), point=(-120.740135, 47.751076))

In [4]:
# Download dataset
dataset.download()

In [5]:
# Load downloded data as a dataframe
df = dataset.load()
df.head()

,datetime,geometry,dayl,prcp,srad,swe,tmax,tmin,vp
0,1990-01-01,POINT (-120.74013 47.75108),29818.61,12.68,86.56,66.10,2.14,-2.68,501.57
1,1990-01-02,POINT (-120.74013 47.75108),29875.77,10.44,114.20,66.00,1.47,-5.76,397.66
2,1990-01-03,POINT (-120.74013 47.75108),29937.57,7.58,98.15,65.27,1.33,-4.27,445.06
3,1990-01-04,POINT (-120.74013 47.75108),30003.96,15.35,111.24,63.49,5.19,-1.76,537.01
4,1990-01-05,POINT (-120.74013 47.75108),30074.88,11.16,109.80,61.99,4.23,-2.43,510.71


### Data at a single point and single variable


In [ ]:
# The latitude of Washington, the USA is 47.751076, and the longitude is -120.740135
target_lat = 47.751076
target_lon = -120.740135

In [6]:
# Create a dataset instance
dataset = DaymetSinglePoint(
    point=(target_lon, target_lat), years=[1990, 2020], variables=("tmin",)
)
dataset

DaymetSinglePoint(dataset='daymet_single_point', years=YearRange(start=1990, end=2020), variables=('tmin',), point=(-120.740135, 47.751076))

In [7]:
# Download dataset
dataset.download()

In [8]:
# Load downloded data as a dataframe
df_wa_point = dataset.load()
# df_wa_point.head()
df_wa_point

,datetime,geometry,tmin
0,1990-01-01,POINT (-120.74013 47.75108),-2.68
1,1990-01-02,POINT (-120.74013 47.75108),-5.76
2,1990-01-03,POINT (-120.74013 47.75108),-4.27
3,1990-01-04,POINT (-120.74013 47.75108),-1.76
4,1990-01-05,POINT (-120.74013 47.75108),-2.43
...,...,...,...
11310,2020-12-26,POINT (-120.74013 47.75108),-5.66
11311,2020-12-27,POINT (-120.74013 47.75108),-3.84
11312,2020-12-28,POINT (-120.74013 47.75108),-4.36
11313,2020-12-29,POINT (-120.74013 47.75108),-4.93


### Data from a bounding box


In [ ]:
# Create a bounding box
# top left / bottom right pair (lat,lon,lat,lon)
area = {"name": "indianapolis", "bbox": [-86.5, 39.5, -86, 40.1]}

# Create an instance of dataset
dataset = DaymetBoundingBox(area=area, years=[2019, 2020])
dataset

DaymetBoundingBox(dataset='daymet_bounding_box', years=YearRange(start=2019, end=2020), area=NamedArea(name='indianapolis', bbox=BoundingBox(xmin=-86.5, ymin=39.5, xmax=-86.0, ymax=40.1)), mosaic='na', variables=('dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp'), frequency='daily')

In [ ]:
# Download data
dataset.download()

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


Function call took more than 120 seconds, retrying


  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |=====================================

In [ ]:
# Load data as a xarray.Dataset
xa_wa = dataset.load()
xa_wa

<xarray.Dataset>
Dimensions:                  (y: 70, x: 52, time: 1095)
Coordinates:
  * y                        (y) float32 -159.0 -160.0 -161.0 ... -227.0 -228.0
  * x                        (x) float32 1.095e+03 1.096e+03 ... 1.146e+03
  * time                     (time) datetime64[ns] 2018-01-01T12:00:00 ... 20...
Data variables:
    lat                      (time, y, x) float32 dask.array<chunksize=(365, 70, 52), meta=np.ndarray>
    lambert_conformal_conic  (time) float64 -3.277e+04 -3.277e+04 ... -3.277e+04
    lon                      (time, y, x) float32 dask.array<chunksize=(365, 70, 52), meta=np.ndarray>
    dayl                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    prcp                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    srad                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    swe                      (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    tmin                     (time, y, x) float32 dask.array<chunksize=(365, 70, 52), meta=np.ndarray>
    vp                       (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
Attributes:
    start_year:          2019
    source:              Daymet Software Version 4.0
    Version_software:    Daymet Software Version 4.0
    Version_data:        Daymet Data Version 4.0
    Conventions:         CF-1.6
    citation:            Please see http://daymet.ornl.gov/ for current Dayme...
    references:          Please see http://daymet.ornl.gov/ for current infor...
    History:             Translated to CF-1.0 Conventions by Netcdf-Java CDM ...
    geospatial_lat_min:  39.43457395273136
    geospatial_lat_max:  40.166069168093486
    geospatial_lon_min:  -86.62665724948563
    geospatial_lon_max:  -85.85920225420271

### Data at multiple points


In [ ]:
# Create an instance of dataset
points = [(-120.740135, 47.751076), (-120.742, 47.76)]
dataset = DaymetMultiplePoints(points=points, years=[2019, 2020])
dataset

DaymetMultiplePoints(dataset='daymet_multiple_points', points=[(-120.740135, 47.751076), (-120.742, 47.76)], years=(2019, 2020))

In [ ]:
# Download data
dataset.download()


R version 4.2.2 (2022-10-31) -- "Innocent and Trusting"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: x86_64-conda-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

>         library(daymetr)
>         daymetr::download_daymet(
+             site = "daymet_single_point_-120.740135_47.751076",
+             lat = 47.751076,
+             lon = -120.740135,
+             start = 2019,
+             end =  2020,
+             path="/tmp/data",
+             intern


Done !



>         
> 

R version 4.2.2 (2022-10-31) -- "Innocent and Trusting"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: x86_64-conda-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

>         library(daymetr)
>         daymetr::download_daymet(
+             site = "daymet_single_point_-120.742_47.76",
+             lat = 47.76,
+             lon = -120.742,
+             start = 2019,
+             end =  2020,
+             path="/tmp/data",
+             intern

>         
> 


Done !



In [ ]:
# Load data as a dataframe
xa_wa = dataset.load()
xa_wa

<xarray.Dataset>
Dimensions:                  (y: 70, x: 52, time: 1095)
Coordinates:
  * y                        (y) float32 -159.0 -160.0 -161.0 ... -227.0 -228.0
  * x                        (x) float32 1.095e+03 1.096e+03 ... 1.146e+03
  * time                     (time) datetime64[ns] 2018-01-01T12:00:00 ... 20...
Data variables:
    lat                      (time, y, x) float32 dask.array<chunksize=(365, 70, 52), meta=np.ndarray>
    lambert_conformal_conic  (time) float64 -3.277e+04 -3.277e+04 ... -3.277e+04
    lon                      (time, y, x) float32 dask.array<chunksize=(365, 70, 52), meta=np.ndarray>
    dayl                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    prcp                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    srad                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    swe                      (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
    tmin                     (time, y, x) float32 dask.array<chunksize=(365, 70, 52), meta=np.ndarray>
    vp                       (time, y, x) float32 dask.array<chunksize=(730, 70, 52), meta=np.ndarray>
Attributes:
    start_year:          2019
    source:              Daymet Software Version 4.0
    Version_software:    Daymet Software Version 4.0
    Version_data:        Daymet Data Version 4.0
    Conventions:         CF-1.6
    citation:            Please see http://daymet.ornl.gov/ for current Dayme...
    references:          Please see http://daymet.ornl.gov/ for current infor...
    History:             Translated to CF-1.0 Conventions by Netcdf-Java CDM ...
    geospatial_lat_min:  39.43457395273136
    geospatial_lat_max:  40.166069168093486
    geospatial_lon_min:  -86.62665724948563
    geospatial_lon_max:  -85.85920225420271